In [2]:
!pip install torch torchaudio audiocraft numpy scipy scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 14.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 7.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of audiocraft to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.4/615.4 kB 38.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 610.4/610.4 kB 54.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 75.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [40]:
import numpy as np
import torch
import torch.nn as nn
import torchaudio
from torchaudio.transforms import MelSpectrogram
from audiocraft.models import musicgen

# ======================
# 1. EEG PREDICTION MODEL
# ======================
class MusicEEGPredictor(nn.Module):
    def __init__(self):
        super().__init__()
        self.melspec = MelSpectrogram(
            sample_rate=44100,
            n_fft=2048,
            n_mels=64,
            hop_length=512
        )
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1,1))
        )
        self.head = nn.Linear(32, 5)  # 5 EEG bands

    def forward(self, audio):
        # audio shape: [batch, channels, samples]
        specs = self.melspec(audio)  # [batch, n_mels, time]
        specs = specs.unsqueeze(1)  # [batch, 1, n_mels, time]
        features = self.encoder(specs).squeeze()
        return torch.sigmoid(self.head(features))

In [41]:
# ======================
# 2. THERAPY SYSTEM
# ======================
class ClinicalMusicTherapySystem:
    def __init__(self):
        self.model = musicgen.MusicGen.get_pretrained('facebook/musicgen-melody')
        self.model.set_generation_params(duration=30)
        self.eeg_predictor = MusicEEGPredictor().eval()
        self.thresholds = {
            'gsr_high': 0.7,
            'hrv_low': 30,
            'eeg_alpha_min': 0.6
        }

    def calculate_hrv(self, rr_intervals):
        diffs = np.diff(rr_intervals)
        return np.sqrt(np.mean(diffs**2))

    def physiological_feedback_loop(self, bio_data):
        gsr_factor = min(1.0, bio_data['gsr'].mean() / self.thresholds['gsr_high'])
        hrv = self.calculate_hrv(bio_data['rr_intervals'])
        hrv_factor = hrv / self.thresholds['hrv_low']

        return {
            'tempo': max(40, min(160, int(90 * (1 - 0.3*gsr_factor + 0.1*hrv_factor)))),
            'brightness': max(0.2, min(1.0, 1.0 - gsr_factor)),
            'complexity': max(0.1, min(0.9, 0.3 + hrv_factor))
        }

    def simulate_music_response(self, audio_waveform):
        audio_tensor = torch.from_numpy(audio_waveform).float().unsqueeze(0)
        with torch.no_grad():
            eeg_pred = self.eeg_predictor(audio_tensor)
        return {
            'delta': eeg_pred[0].item(),
            'theta': eeg_pred[1].item(),
            'alpha': eeg_pred[2].item(),
            'beta': eeg_pred[3].item(),
            'gamma': eeg_pred[4].item()
        }

    def generate_therapeutic_music(self, bio_data):
        adjustments = self.physiological_feedback_loop(bio_data)
        descriptors = [
            f"Piano in C major, {adjustments['tempo']} BPM",
            f"Brightness: {adjustments['brightness']:.1f}",
            "Simple structure" if adjustments['complexity'] < 0.5 else "Complex"
        ]
        audio = self.model.generate(descriptions=descriptors)
        return audio, adjustments


In [42]:
# ======================
# 3. USAGE
# ======================
if __name__ == "__main__":
    therapy_system = ClinicalMusicTherapySystem()

    mock_bio_data = {
        'gsr': np.random.uniform(0.3, 0.9, 100),
        'rr_intervals': np.random.normal(800, 50, 100),
        'eeg': {
            'F3_alpha': 0.7,
            'F4_alpha': 0.5,
            'alpha': np.random.uniform(0.4, 0.8, 100),
            'beta': np.random.uniform(0.3, 0.7, 100)
        }
    }

    audio, adjustments = therapy_system.generate_therapeutic_music(mock_bio_data)
    print(f"Generated music with adjustments: {adjustments}")

/usr/local/lib/python3.11/dist-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Generated music with adjustments: {'tempo': 88, 'brightness': 0.2, 'complexity': 0.9}


In [6]:
import torch
import torch.nn as nn
from torchaudio.transforms import MelSpectrogram

class MusicEEGPredictor(nn.Module):
    def __init__(self):
        super().__init__()
        self.melspec = MelSpectrogram(
            sample_rate=44100,
            n_fft=2048,
            n_mels=64,
            hop_length=512
        )
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU()
        )
        self.lstm = nn.LSTM(input_size=32, hidden_size=64, batch_first=True)
        self.head = nn.Linear(64, 5)  # EEG bands

    def forward(self, audio):
        specs = self.melspec(audio)  # [B, M, T]
        specs = specs.unsqueeze(1)  # [B, 1, M, T]
        features = self.cnn(specs)  # [B, C, M', T']
        B, C, M, T = features.size()
        features = features.permute(0, 3, 1, 2).reshape(B, T, -1)  # [B, T, features]
        lstm_out, _ = self.lstm(features)
        out = self.head(lstm_out[:, -1, :])  # Last time step
        return torch.sigmoid(out)


In [7]:
from audiocraft.models import musicgen

class MusicGenAdapter:
    def __init__(self, duration=30):
        self.model = musicgen.MusicGen.get_pretrained('facebook/musicgen-melody')
        self.model.set_generation_params(duration=duration)

    def generate_music(self, mood_descriptor):
        audio = self.model.generate(descriptions=[mood_descriptor])
        return audio


In [8]:
import numpy as np
import torch

class ClinicalMusicTherapySystem:
    def __init__(self):
        self.music_gen = MusicGenAdapter()
        self.eeg_predictor = MusicEEGPredictor().eval()
        self.thresholds = {
            'gsr_high': 0.7,
            'hrv_low': 30,
        }

    def calculate_hrv(self, rr_intervals):
        rr_diff = np.diff(rr_intervals)
        return np.sqrt(np.mean(rr_diff ** 2))

    def physiological_feedback_loop(self, bio_data):
        gsr_mean = np.mean(bio_data['gsr'])
        hrv = self.calculate_hrv(bio_data['rr_intervals'])

        gsr_factor = min(1.0, gsr_mean / self.thresholds['gsr_high'])
        hrv_factor = hrv / self.thresholds['hrv_low']

        tempo = int(90 * (1 - 0.3 * gsr_factor + 0.1 * hrv_factor))
        brightness = 1.0 - gsr_factor
        complexity = 0.3 + hrv_factor

        return {
            'tempo': max(40, min(160, tempo)),
            'brightness': max(0.2, min(1.0, brightness)),
            'complexity': max(0.1, min(0.9, complexity)),
            'hrv': hrv,
            'gsr': gsr_mean
        }

    def simulate_eeg_response(self, audio_waveform):
        audio_tensor = torch.from_numpy(audio_waveform).float().unsqueeze(0)
        with torch.no_grad():
            eeg_pred = self.eeg_predictor(audio_tensor)
        return dict(zip(['delta', 'theta', 'alpha', 'beta', 'gamma'], eeg_pred.squeeze().tolist()))

    def generate_music_with_feedback(self, bio_data):
        feedback = self.physiological_feedback_loop(bio_data)
        descriptor = f"Relaxing piano, {feedback['tempo']} BPM, brightness {feedback['brightness']:.2f}, " \
                     f"{'complex structure' if feedback['complexity'] > 0.5 else 'simple melody'}"
        audio = self.music_gen.generate_music(descriptor)
        return audio, feedback, descriptor


In [9]:
import json
from datetime import datetime

if __name__ == "__main__":
    therapy = ClinicalMusicTherapySystem()

    mock_bio_data = {
        'gsr': np.random.uniform(0.3, 0.9, 100),
        'rr_intervals': np.random.normal(800, 50, 100)
    }

    music, feedback, descriptor = therapy.generate_music_with_feedback(mock_bio_data)

    log_data = {
        'timestamp': datetime.now().isoformat(),
        'bio_feedback': feedback,
        'music_descriptor': descriptor
    }

    with open("therapy_log.json", "a") as log_file:
        log_file.write(json.dumps(log_data) + "\n")

    print("Music generated and logged:")
    print(descriptor)


/usr/local/lib/python3.11/dist-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Music generated and logged:
Relaxing piano, 90 BPM, brightness 0.20, complex structure


In [16]:
import os
import pandas as pd
import numpy as np
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler

class GameemoEEGDataset(Dataset):
    def __init__(self, root_dir, channels=None, max_samples=5000):
        """
        root_dir: path to root GameEmo folder with S01, S02, ... subfolders
        channels: list of channel indices to select (None = all)
        max_samples: max length of EEG signal per trial (pad/cut to this length)
        """
        self.root_dir = root_dir
        self.samples = []
        self.labels = []

        subjects = [f for f in os.listdir(root_dir) if f.startswith('S')]
        print(f"Subjects found: {subjects}")

        for subj in subjects:
            subj_path = os.path.join(root_dir, subj)
            eeg_folder = os.path.join(subj_path, "Preproceed EEG Data", "CSV")
            sam_folder = os.path.join(subj_path, "Sam Ratings")

            sam_file = None
            for f in os.listdir(sam_folder):
                if f.endswith(".csv"):
                    sam_file = os.path.join(sam_folder, f)
                    break
            if sam_file is None:
                print(f"No SAM file for {subj}, skipping...")
                continue

            sam_df = pd.read_csv(sam_file)
            # Ensure 'Valence' column exists in sam_df
            if 'Valence' not in sam_df.columns:
                raise ValueError(f"Valence column not found in SAM file for {subj}")

            # For standardization
            scaler = StandardScaler()

            # Load all EEG files for subject
            eeg_files = sorted([f for f in os.listdir(eeg_folder) if f.endswith('.csv')])

            for i, eeg_file in enumerate(eeg_files):
                eeg_path = os.path.join(eeg_folder, eeg_file)
                eeg_data = pd.read_csv(eeg_path).values  # shape [samples, channels]

                # Select channels if specified
                if channels is not None:
                    eeg_data = eeg_data[:, channels]

                # Pad or truncate to max_samples
                if eeg_data.shape[0] > max_samples:
                    eeg_data = eeg_data[:max_samples, :]
                else:
                    pad_len = max_samples - eeg_data.shape[0]
                    eeg_data = np.pad(eeg_data, ((0, pad_len), (0, 0)), 'constant')

                # Standardize per trial
                eeg_data = scaler.fit_transform(eeg_data)

                # Flatten or keep 2D: here keep 2D (channels × samples)
                # Transpose to (channels, samples)
                eeg_data = eeg_data.T.astype(np.float32)

                # Match SAM label
                # Trial indices should correspond; adjust if mismatch occurs
                valence = sam_df.iloc[i]['Valence']

                self.samples.append(eeg_data)
                self.labels.append(valence)

        print(f"Loaded {len(self.samples)} trials from {len(subjects)} subjects.")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx], self.labels[idx]


In [17]:
import torch
import torch.nn as nn

class MusicEEGPredictor(nn.Module):
    def __init__(self, n_channels=32, n_samples=5000):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv1d(n_channels, 64, kernel_size=7, stride=2, padding=3),
            nn.ReLU(),
            nn.MaxPool1d(4),
            nn.Conv1d(64, 128, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(1)
        )
        self.head = nn.Linear(128, 1)  # Regression output (valence)

    def forward(self, x):
        # x shape: [batch, channels, samples]
        features = self.encoder(x)  # [batch, 128, 1]
        features = features.squeeze(-1)  # [batch, 128]
        out = self.head(features)  # [batch, 1]
        return out.squeeze(1)  # [batch]


In [18]:
import torch.optim as optim
from torch.utils.data import DataLoader

def train_model(dataset, epochs=10, batch_size=16, device='cpu'):
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    model = MusicEEGPredictor(n_channels=dataset[0][0].shape[0], n_samples=dataset[0][0].shape[1])
    model.to(device)

    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        for signals, labels in dataloader:
            signals = signals.to(device)
            labels = labels.to(device).float()

            optimizer.zero_grad()
            outputs = model(signals)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * signals.size(0)

        epoch_loss = running_loss / len(dataset)
        print(f"Epoch {epoch+1}/{epochs} - Loss: {epoch_loss:.4f}")

    return model


In [39]:
import json
from datetime import datetime
import matplotlib.pyplot as plt

class ClinicalMusicTherapySystem:
    def __init__(self, eeg_model, device='cpu'):
        self.device = device
        self.model = musicgen.MusicGen.get_pretrained('facebook/musicgen-melody')
        self.model.set_generation_params(duration=30)
        self.eeg_predictor = eeg_model.to(device).eval()

    def generate_music(self, bio_data):
        tempo = 90  # could be modified by bio_data
        descriptor = f"Piano in C major, {tempo} BPM"
        audio = self.model.generate(descriptions=[descriptor])
        return audio, descriptor

    def simulate_eeg_response(self, audio_waveform):
        audio_tensor = torch.from_numpy(audio_waveform).float()
        if audio_tensor.dim() == 1:
            audio_tensor = audio_tensor.unsqueeze(0).unsqueeze(0)  # [1, 1, samples]
        elif audio_tensor.dim() == 2:
            audio_tensor = audio_tensor.unsqueeze(1)

        audio_tensor = audio_tensor.to(self.device)
        with torch.no_grad():
            eeg_pred = self.eeg_predictor(audio_tensor)
        return eeg_pred.cpu().numpy()

    def log_session(self, bio_data, descriptor, eeg_pred, filepath="therapy_log.json"):
        session = {
            'timestamp': datetime.now().isoformat(),
            'bio_data': bio_data,
            'music_description': descriptor,
            'eeg_prediction': eeg_pred.tolist() if hasattr(eeg_pred, 'tolist') else eeg_pred
        }
        with open(filepath, 'a') as f:
            json.dump(session, f)
            f.write('\n')

    def plot_trends(self, log_file="therapy_log.json"):
        timestamps, eeg_vals, tempos = [], [], []
        with open(log_file) as f:
            for line in f:
                session = json.loads(line)
                timestamps.append(session['timestamp'])
                eeg_vals.append(session['eeg_prediction'])
                # Extract tempo from description
                desc = session['music_description']
                tempo = int(desc.split(",")[1].strip().split(" ")[0])
                tempos.append(tempo)

        plt.figure(figsize=(10,5))
        plt.plot(timestamps, eeg_vals, label="EEG Prediction (Valence)")
        plt.xlabel("Time")
        plt.ylabel("Valence")
        plt.title("EEG Valence Trend During Therapy Sessions")
        plt.xticks(rotation=45)
        plt.legend()
        plt.tight_layout()
        plt.show()


In [ ]:
import kagglehub

if __name__ == "__main__":
    dataset_path = kagglehub.dataset_download("birdy654/eeg-brainwave-dataset-feeling-emotions")
    dataset = GameemoEEGDataset(dataset_path, channels=list(range(32)), max_samples=5000)

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = train_model(dataset, epochs=10, batch_size=16, device=device)

    therapy = ClinicalMusicTherapySystem(model, device=device)

    # Simulate bio_data
    bio_data = {
        'gsr': np.random.uniform(0.3, 0.9, 100).tolist(),
        'rr_intervals': np.random.normal(800, 50, 100).tolist()
    }

    audio, desc = therapy.generate_music(bio_data)
    dummy_audio = np.random.randn(44100 * 30).astype(np.float32)
    eeg_pred = therapy.simulate_eeg_response(dummy_audio)

    print(f"Music descriptor: {desc}")
    print(f"EEG Prediction: {eeg_pred}")

    therapy.log_session(bio_data, desc, eeg_pred)
    therapy.plot_trends()